In [23]:
# Load standard modules
import csv
import numpy as np
from matplotlib import pyplot as plt

# Load tudatpy modules
from tudatpy.interface import spice
from tudatpy import numerical_simulation
from tudatpy.numerical_simulation import environment
from tudatpy.numerical_simulation import environment_setup, propagation_setup, estimation_setup
from tudatpy.astro import element_conversion
from tudatpy import constants
from tudatpy.util import result2array
from tudatpy.astro.time_conversion import DateTime

In [24]:
# Load spice kernels
spice.load_standard_kernels()

# Set simulation start and end epochs
simulation_start_epoch = DateTime(2000, 1, 1).epoch()
simulation_end_epoch   = DateTime(2000, 1, 1, 18).epoch()

In [25]:
# Create default body settings for "Earth"
bodies_to_create = ["Earth"]

# Create default body settings for bodies_to_create, with "Earth"/"J2000" as the global frame origin and orientation
global_frame_origin = "Earth"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)

# Create the system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [26]:
# Create vehicle objects.
bodies.create_empty_body("Chief")
bodies.create_empty_body("Deputy1")
bodies.create_empty_body("Deputy2")
bodies.create_empty_body("Deputy3")

# Set mass of satellites
bodies.get("Chief").mass = 1.0
bodies.get("Deputy1").mass = 1.0
bodies.get("Deputy2").mass = 1.0
bodies.get("Deputy3").mass = 1.0

# Create aerodynamic coefficient interface settings
reference_area = 1e-4
drag_coefficient = 2.22
aero_coefficient_settings = environment_setup.aerodynamic_coefficients.constant(
    reference_area, [drag_coefficient, 0.0, 0.0]
)

# Add the aerodynamic interface to the environment
environment_setup.add_aerodynamic_coefficient_interface(
            bodies, "Chief", aero_coefficient_settings)
environment_setup.add_aerodynamic_coefficient_interface(
            bodies, "Deputy1", aero_coefficient_settings)
environment_setup.add_aerodynamic_coefficient_interface(
            bodies, "Deputy2", aero_coefficient_settings)
environment_setup.add_aerodynamic_coefficient_interface(
            bodies, "Deputy3", aero_coefficient_settings)

In [27]:
# Define bodies that are propagated
bodies_to_propagate = ["Chief", "Deputy1", "Deputy2", "Deputy3"]

# Define central bodies of propagation
central_bodies = ["Earth", "Earth", "Earth", "Earth"]

In [28]:
# Define unique (Earth) accelerations acting on each vehicle
accelerations_settings = dict(
    Earth=[
        propagation_setup.acceleration.spherical_harmonic_gravity(2, 0),
        propagation_setup.acceleration.aerodynamic()
    ])


# Create global accelerations settings dictionary
acceleration_settings = {"Chief": accelerations_settings,
                         "Deputy1": accelerations_settings,
                         "Deputy2": accelerations_settings,
                         "Deputy3": accelerations_settings}
                         

# Create acceleration models
acceleration_models = propagation_setup.create_acceleration_models(
    bodies,
    acceleration_settings,
    bodies_to_propagate,
    central_bodies)

In [29]:
# Retrieve Earth's gravitational parameter
earth_gravitational_parameter = bodies.get("Earth").gravitational_parameter

# Convert keplerian to cartesian elements
initial_state_chief = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=earth_gravitational_parameter,
    semi_major_axis=6978e3,
    eccentricity=2.6e-6,
    inclination=np.deg2rad(97.79),
    argument_of_periapsis=np.deg2rad(303.34),
    longitude_of_ascending_node=np.deg2rad(1.5e-5),
    true_anomaly=np.deg2rad(157.36),
)

initial_state_deputy1 = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=earth_gravitational_parameter,
    semi_major_axis=6978e3,
    eccentricity=6.48e-3,
    inclination=np.deg2rad(97.26),
    argument_of_periapsis=np.deg2rad(281.15),
    longitude_of_ascending_node=np.deg2rad(272.80),
    true_anomaly=np.deg2rad(269.52),
)

initial_state_deputy2 = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=earth_gravitational_parameter,
    semi_major_axis=6978e3,
    eccentricity=6.6e-5,
    inclination=np.deg2rad(97.79),
    argument_of_periapsis=np.deg2rad(104.07),
    longitude_of_ascending_node=np.deg2rad(149.99),
    true_anomaly=np.deg2rad(206.00),
)

initial_state_deputy3 = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=earth_gravitational_parameter,
    semi_major_axis=6978e3,
    eccentricity=1.3e-5,
    inclination=np.deg2rad(97.79),
    argument_of_periapsis=np.deg2rad(257.43),
    longitude_of_ascending_node=np.deg2rad(70.00),
    true_anomaly=np.deg2rad(332.57),
)

# Create initial state
initial_states = np.concatenate((initial_state_chief, initial_state_deputy1, initial_state_deputy2, initial_state_deputy3))

In [30]:
# Create termination settings
termination_condition = propagation_setup.propagator.time_termination(simulation_end_epoch)

# Create numerical integrator settings
fixed_step_size = 60.0
integrator_settings = propagation_setup.integrator.runge_kutta_4(fixed_step_size)

# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_states,
    simulation_start_epoch,
    integrator_settings,
    termination_condition
)

In [31]:
# Setup parameters settings to propagate the state transition matrix
parameter_settings = estimation_setup.parameter.initial_states(propagator_settings, bodies)

# Create the parameters that will be estimated
parameters_to_estimate = estimation_setup.create_parameter_set(parameter_settings, bodies)

In [32]:
# Create the variational equation solver and propagate the dynamics
variational_equations_solver = numerical_simulation.create_variational_equations_solver(
    bodies, propagator_settings, parameters_to_estimate, simulate_dynamics_on_creation=True
)

# Extract the resulting state history, state transition matrix history, and sensitivity matrix history
states = variational_equations_solver.state_history
state_transition_matrices = variational_equations_solver.state_transition_matrix_history

In [33]:
with open('../data/data_tudatpy/data_tudatpy_x_true.csv', 'w', newline='') as csvfile:
    fieldnames = ['x_chief', 'y_chief', 'z_chief', 'x_deputy1', 'y_deputy1', 'z_deputy1', 'x_deputy2', 'y_deputy2', 'z_deputy2', 'x_deputy3', 'y_deputy3', 'z_deputy3']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for values in states.values():
        writer.writerow({
            'x_chief': values[0] * 1e-3, 'y_chief': values[1] * 1e-3, 'z_chief': values[2] * 1e-3,
            'x_deputy1': values[6] * 1e-3, 'y_deputy1': values[7] * 1e-3, 'z_deputy1': values[8] * 1e-3,
            'x_deputy2': values[12] * 1e-3, 'y_deputy2': values[13] * 1e-3, 'z_deputy2': values[14] * 1e-3,
            'x_deputy3': values[18] * 1e-3, 'y_deputy3': values[19] * 1e-3, 'z_deputy3': values[20] * 1e-3,
        })